In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


# Dataset:

In [2]:
import kagglehub

path = kagglehub.dataset_download("yudhaislamisulistya/plants-type-datasets")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'plants-type-datasets' dataset.
Path to dataset files: /kaggle/input/plants-type-datasets


In [3]:
import os

base_dir = "/kaggle/input/plants-type-datasets"

print("Contents of dataset root:")
print(os.listdir(base_dir))

Contents of dataset root:
['split_ttv_dataset_type_of_plants']


In [4]:
split_dir = "/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants"

print("Train / Validation / Test folders:")
print(os.listdir(split_dir))

Train / Validation / Test folders:
['Test_Set_Folder', 'Validation_Set_Folder', 'Train_Set_Folder']


#DataLoaders:

## Transformations

In [5]:
from torchvision import transforms

image_size = 64
train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

val_test_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])


## Creating Image Datasets

In [6]:
train_dir = "/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Train_Set_Folder"
val_dir   = "/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Validation_Set_Folder"
test_dir  = "/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Test_Set_Folder"

In [7]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder(root=train_dir, transform=train_transforms)
val_dataset   = ImageFolder(root=val_dir, transform=val_test_transforms)
test_dataset  = ImageFolder(root=test_dir, transform=val_test_transforms)

num_classes = len(train_dataset.classes)
print("Number of classes:", num_classes)


Number of classes: 30


## DataLoaders

In [8]:
from torch.utils.data import DataLoader

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Train batches:", len(train_loader))
print("Validation batches:", len(val_loader))
print("Test batches:", len(test_loader))


Train batches: 375
Validation batches: 48
Test batches: 47


#Convolutional Neural Network (CNN):

In [9]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.LazyLinear(256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [10]:
model = SimpleCNN(num_classes=30).to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): LazyLinear(in_features=0, out_features=256, bias=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=30, bias=True)
  )
)


#Training Loop:

In [11]:
def training_loop(model, train_loader, val_loader, loss_function, optimizer, num_epochs, device):
    model.to(device)



    for epoch in range(num_epochs):


        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_function(outputs, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss /= len(train_loader.dataset)
        train_acc = 100.0 * correct / total




        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                loss = loss_function(outputs, labels)

                val_loss += loss.item() * images.size(0)

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = 100.0 * correct / total

        print(
            f"Epoch [{epoch+1}/{num_epochs}] | "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%"
        )


In [12]:
num_epochs = 5

training_loop(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_function=loss_function,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device
)

Epoch [1/5] | Train Loss: 2.4872, Train Acc: 24.07% | Val Loss: 2.0081, Val Acc: 37.43%
Epoch [2/5] | Train Loss: 1.8090, Train Acc: 42.70% | Val Loss: 1.6648, Val Acc: 45.51%
Epoch [3/5] | Train Loss: 1.5178, Train Acc: 51.84% | Val Loss: 1.4689, Val Acc: 53.33%
Epoch [4/5] | Train Loss: 1.3199, Train Acc: 57.53% | Val Loss: 1.3245, Val Acc: 58.61%
Epoch [5/5] | Train Loss: 1.1663, Train Acc: 61.59% | Val Loss: 1.1375, Val Acc: 63.30%


#Testing:

In [13]:
def test_model(model, test_loader, loss_function, device):
    model.to(device)
    model.eval()

    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = loss_function(outputs, labels)

            test_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss = test_loss / len(test_loader.dataset)
    test_acc = 100.0 * correct / total

    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")

    return test_loss, test_acc


In [14]:
test_loss, test_acc = test_model(model, test_loader, loss_function, device)

Test Loss: 1.1420 | Test Accuracy: 63.48%
